In [1]:
import pandas as pd
import requests
import json
import os
import getpass
from pathlib import Path
import sqlite3
from time import sleep
import helpful_functions as hf
import ace_lib as ace
import pickle

In [2]:
s = ace.start_session()

Complete biometrics authentication and press any key to continue: 
https://api.worldquantbrain.com/authentication/persona?inquiry=inq_HQXNx3sCUgcChHX1uYUyqrz1EWs4



In [3]:

datafields_df = hf.get_datafields(s, region="USA", universe="TOP3000", delay=1) # by default we load all datasets USA TOP3000 delay 1
datafields_df.head() # DataFrame.head() shows first 5 rows of the dataframe 

,id,description,dataset,category,subcategory,region,delay,universe,type,coverage,userCount,alphaCount,pyramidMultiplier,themes
0,act_12m_bps_value,actual book value per share regarding the last...,"{'id': 'analyst7', 'name': 'Broker Estimates'}","{'id': 'analyst', 'name': 'Analyst'}","{'id': 'analyst-analyst-estimates', 'name': 'A...",USA,1,TOP3000,MATRIX,0.5039,12,12,1.2,[]
1,act_12m_cps_value,actual cash flow per share regarding the last ...,"{'id': 'analyst7', 'name': 'Broker Estimates'}","{'id': 'analyst', 'name': 'Analyst'}","{'id': 'analyst-analyst-estimates', 'name': 'A...",USA,1,TOP3000,MATRIX,0.3406,36,87,1.2,[]
2,act_12m_cpx_value,actual capital expenditure regarding the last ...,"{'id': 'analyst7', 'name': 'Broker Estimates'}","{'id': 'analyst', 'name': 'Analyst'}","{'id': 'analyst-analyst-estimates', 'name': 'A...",USA,1,TOP3000,MATRIX,0.4691,37,72,1.2,[]
3,act_12m_dps_value,actual dividend per share regarding the last 4...,"{'id': 'analyst7', 'name': 'Broker Estimates'}","{'id': 'analyst', 'name': 'Analyst'}","{'id': 'analyst-analyst-estimates', 'name': 'A...",USA,1,TOP3000,MATRIX,0.4161,14,21,1.2,[]
4,act_12m_ebi_value,actual EBIT regarding the last 4 quarters,"{'id': 'analyst7', 'name': 'Broker Estimates'}","{'id': 'analyst', 'name': 'Analyst'}","{'id': 'analyst-analyst-estimates', 'name': 'A...",USA,1,TOP3000,MATRIX,0.7226,83,256,1.2,[]


In [4]:
datafields_df_vector = datafields_df[datafields_df["type"] == "VECTOR"]
datafields_df_matrix = datafields_df[datafields_df["type"] == "MATRIX"]

# Sort datafields by alphaCount in descending order
datafields_df_matrix = datafields_df_matrix.sort_values(by="alphaCount", ascending=False)
datafields_df_vector = datafields_df_vector.sort_values(by="alphaCount", ascending=False)

# Display the length of each datafield type
print(len(datafields_df_vector))
print(len(datafields_df_matrix))

1721
8246


In [8]:

li = datafields_df_matrix.id.values.tolist()
len(li)

8246

In [9]:
expressions = [
   "zscore(group_zscore(ts_backfill({}, 120), sector) - group_zscore(ts_backfill(anl11_2_3pme, 120), sector)) ",
]

In [10]:
count = 0
for i in li:
    for expr in expressions:
        try:
            count = count + 1
            print(count)
            if(count<98) : continue;
            
            simulate_data = {
                "type": "REGULAR",
                "settings": {
                    "instrumentType": "EQUITY",
                    "region": "USA",
                    "universe": "TOP3000",
                    "delay": 1,
                    "decay": 0,
                    "neutralization": "SECTOR",
                    "truncation": 0.08,
                    "pasteurization": "ON",
                    "unitHandling": "VERIFY",
                    "nanHandling": "OFF",
                    "language": "FASTEXPR",
                    "visualization": False,
                    "testPeriod": "P2Y0M"
                },
                "regular": expr.format(i)
            }
            
            simulate_response = s.post('https://api.worldquantbrain.com/simulations', json=simulate_data)
            print(simulate_data)
            print(simulate_response)
    
            
            simulation_progress_url = simulate_response.headers['Location']
            print(simulation_progress_url)
            finished = False
            
            while True:
                simulation_progress = s.get(simulation_progress_url)
              
                print('simulation_progress')
                if simulation_progress.headers.get("Retry-After", 0) == 0:
                    break
                sleep(float(simulation_progress.headers["Retry-After"]))
                
        except Exception as e:
            print(e)
        
 

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
HTTPSConnectionPool(host='api.worldquantbrain.com', port=443): Max retries exceeded with url: /simulations (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001DDEC149850>: Failed to resolve 'api.worldquantbrain.com' ([Errno 11001] getaddrinfo failed)"))
99
HTTPSConnectionPool(host='api.worldquantbrain.com', port=443): Max retries exceeded with url: /simulations (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001DDEEA86ED0>: Failed to resolve 'api.worldquantbrain.com' ([Errno 11001] getaddrinfo failed)"))
100
HTTPSConnectionPool(host='api.worldquantbrain.com', port=443): Max retries exceeded with url: /simulations (Caused by NameResolution

KeyboardInterrupt: 